<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/RTAS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
!pip uninstall opencv-python

In [97]:
!pip uninstall opencv-contrib-python

Found existing installation: opencv-contrib-python 4.5.5.64
Uninstalling opencv-contrib-python-4.5.5.64:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python-4.5.5.64.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python.libs/libQt5Core-39545cc7.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python.libs/libQt5Gui-ba0a2070.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python.libs/libQt5Test-c38a5234.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python.libs/libQt5Widgets-e69d94fb.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python.libs/libQt5XcbQpa-ca221f44.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python.libs/libX11-xcb-69166bdf.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python.libs/libXau-00ec42fe.so.6.0.0
    /usr/local/lib/python3.7/dist-packages/ope

In [98]:
!pip install opencv-contrib-python

  Using cached opencv_contrib_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (66.7 MB)


In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pylab import *
from numpy import *
from scipy import linalg
import pandas as pd

In [3]:
k = [[1698.873755, 0.000000, 971.7497705],
     [0.000000, 1698.8796645, 647.7488275],
     [0.000000, 0.000000, 1.000000]]

k = np.asarray(k)

In [4]:
k

array([[1.69887376e+03, 0.00000000e+00, 9.71749770e+02],
       [0.00000000e+00, 1.69887966e+03, 6.47748827e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [5]:
k_inv = np.linalg.inv(k)

In [6]:
k_inv

array([[ 5.88625257e-04,  0.00000000e+00, -5.71996458e-01],
       [ 0.00000000e+00,  5.88623209e-04, -3.81279994e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [7]:
img1 = cv2.imread('/content/im1.jpg', 0)
img2 = cv2.imread('/content/im2.jpg', 0)

In [8]:
def find_correspondences(img1, img2):
  sift = cv2.xfeatures2d.SIFT_create()
  kp1, des1 = sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  flann = cv2.FlannBasedMatcher(dict(algorithm = 0, trees = 5),dict(checks=50))
  matches = flann.knnMatch(des1,des2,k=2)
  pts1 = []
  pts2 = []

  # ratio test as per Lowe's paper
  j = 0
  for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        pts2.append((kp2[m.trainIdx].pt, 1))
        pts1.append(kp1[m.queryIdx].pt)
        j = j+1
        if j == 100:
          break
  pts1 = [list(tup)+[1] for tup in pts1]
  pts2 = [list(tup)+[1] for tup in pts2]
  pts1, pts2 = np.asarray(pts1), np.asarray(pts2)
  pts1 = np.float64(pts1)
  pts2 = np.float64(pts1)

  return pts1, pts2

In [9]:
pts1, pts2 = find_correspondences(img1, img2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [10]:
def convert_to_camera_coordinate_system(pts1, pts2, k):
  k_inv = np.linalg.inv(k)
  pts1 = [np.dot(k_inv, item) for item in pts1]
  pts2 = [np.dot(k_inv, item) for item in pts2]
  return pts1, pts2

In [11]:
def compute_essential(pts1, pts2):
  
  x1, x2 = convert_to_camera_coordinate_system(pts1, pts2, k)
  x1 = np.asarray(x1)
  x2 = np.asarray(x2)
  n = x1.shape[1]
  if x2.shape[1] != n:
    raise ValueError("Number of points don't match.")
 
  # build matrix for equations
  A = []
  for i in range(8):
      z = np.kron(pts2[i], pts1[i])
      A.append(z)
  A = np.asarray(A)
  #print(A.shape)

  U,S,V = linalg.svd(A)
  E = V[-1].reshape(3,3)
  # constrain F
  # make rank 2 by zeroing out last singular value
  Ua,Sa,Va = linalg.svd(E)
  
  sigma = (Sa[0]+Sa[1])/2
  Sa[0] = sigma
  Sa[1] = sigma
  Sa[2] = 0
  
  Ea = np.dot(Ua,np.dot(diag(Sa),Va))
  

  return sigma , Ea

In [12]:
sigma, E = compute_essential(pts1, pts2)

In [13]:
E

array([[-3.43233757e-22,  1.80399143e-08, -3.09152220e-06],
       [-1.80399142e-08, -1.65426508e-18,  7.07106781e-01],
       [ 3.09152221e-06, -7.07106781e-01, -7.18875573e-20]])

In [14]:
def compute_R_and_T(E, sigma):

  U,S,V = linalg.svd(E)

  R_z = [[0, -1, 0],
         [1, 0, 0],
         [0, 0, 1]]
  R_z = np.asarray(R_z)
             
  R1 = np.dot(U ,np.dot(R_z, V))
  R2 = np.dot(U ,np.dot(np.transpose(R_z), V))
  t1 = U[:,2]
  t2 = -U[:,2]

  return R1, R2, t1, t2

In [15]:
R1, R2, t1, t2 = compute_R_and_T(E, sigma)

In [16]:
def calculate_M_and_N_matrix(R1, R2, t1, t2):
  I_O = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]]
  M = np.dot(k, I_O)
  t1 = np.expand_dims(t1, axis = 1)
  R_t = np.hstack((R1, t1))
  N = np.dot(k, R_t)

  return M, N

In [17]:
M, N = calculate_M_and_N_matrix(R1, R2, t1, t2)

In [18]:
def triangulate_point(x1,x2,P1,P2):

        
    A = zeros((6,6))
    A[:3,:4] = P1
    A[3:,:4] = P2
    A[:3,4] = -x1
    A[3:,5] = -x2

    U,S,V = linalg.svd(A)
    X = V[-1,:4]

    return X / X[3]


def triangulate(pts1,pts2,P1,P2):
    
    n = pts1[0].reshape(3,1).shape[1]
    three_d_points = []    
    for j in range(len(pts1)):
      x1 = pts1[j].reshape(3,1)
      x2 = pts2[j].reshape(3,1)
      X = [ triangulate_point(x1[:,i],x2[:,i],P1,P2) for i in range(n)]
      three_d_points.append(array(X).T)


    return three_d_points

In [19]:
ff = triangulate(pts1,pts2,M,N)